# Advanced Copper Market Analysis (Corrected)

Multi-dimensional analysis with properly formatted date axes and corrected JSON structure.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec
import seaborn as sns
from scipy import stats
import warnings

# Add project root to Python path
project_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.insert(0, project_root)

from config.database_config import get_connection_string

warnings.filterwarnings('ignore')

# Enhanced style settings
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['figure.titlesize'] = 15
plt.rcParams['figure.dpi'] = 120
plt.rcParams['savefig.dpi'] = 200

# Professional color palette
COLORS = {
    'primary': '#1f77b4',
    'secondary': '#ff7f0e', 
    'tertiary': '#2ca02c',
    'quaternary': '#d62728',
    'quinary': '#9467bd',
    'dark': '#2c3e50',
    'light': '#ecf0f1'
}

# Function to format date axes properly
def format_date_axis(ax, data_range_days):
    """Format date axis based on data range"""
    if data_range_days <= 30:
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
    elif data_range_days <= 90:
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
    elif data_range_days <= 365:
        ax.xaxis.set_major_locator(mdates.MonthLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    else:
        ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

print("Advanced analysis libraries loaded with date formatting fixes")

## Data Loading and Preprocessing

In [ ]:
def load_comprehensive_data(conn, days=180):
    """Load comprehensive futures data for analysis"""
    query = f"""
    SELECT 
        p.TradeDate,
        m.MetalCode,
        t.TenorTypeName,
        p.SettlementPrice,
        p.Volume,
        p.OpenInterest,
        CASE 
            WHEN t.TenorTypeName LIKE 'Generic 1st%' THEN 1
            WHEN t.TenorTypeName LIKE 'Generic 2nd%' THEN 2
            WHEN t.TenorTypeName LIKE 'Generic 3rd%' THEN 3
            WHEN t.TenorTypeName LIKE 'Generic 4th%' THEN 4
            WHEN t.TenorTypeName LIKE 'Generic 5th%' THEN 5
            WHEN t.TenorTypeName LIKE 'Generic 6th%' THEN 6
            WHEN t.TenorTypeName LIKE 'Generic 7th%' THEN 7
            WHEN t.TenorTypeName LIKE 'Generic 8th%' THEN 8
            WHEN t.TenorTypeName LIKE 'Generic 9th%' THEN 9
            WHEN t.TenorTypeName LIKE 'Generic 10th%' THEN 10
            WHEN t.TenorTypeName LIKE 'Generic 11th%' THEN 11
            WHEN t.TenorTypeName LIKE 'Generic 12th%' THEN 12
            ELSE NULL
        END as TenorNumber
    FROM T_CommodityPrice p
    INNER JOIN M_Metal m ON p.MetalID = m.MetalID
    INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
    WHERE 
        m.MetalCode = 'COPPER'
        AND p.TradeDate >= DATEADD(day, -{days}, GETDATE())
        AND p.SettlementPrice IS NOT NULL
    ORDER BY p.TradeDate DESC, t.TenorTypeName
    """
    
    df = pd.read_sql(query, conn)
    df['TradeDate'] = pd.to_datetime(df['TradeDate'])
    df = df.dropna(subset=['TenorNumber'])
    return df

# Connect and load data
conn = pyodbc.connect(get_connection_string())
print("Connected to database")

# Load 6 months of data
df = load_comprehensive_data(conn, days=180)
print(f"Loaded {len(df):,} records covering {df['TradeDate'].nunique()} trading days")
print(f"Date range: {df['TradeDate'].min().date()} to {df['TradeDate'].max().date()}")

# Calculate data range for formatting
data_range_days = (df['TradeDate'].max() - df['TradeDate'].min()).days
print(f"Data spans {data_range_days} days")

# Create pivot tables
price_pivot = df.pivot_table(
    values='SettlementPrice', index='TradeDate', columns='TenorNumber', aggfunc='mean'
).sort_index()

volume_pivot = df.pivot_table(
    values='Volume', index='TradeDate', columns='TenorNumber', aggfunc='mean'
).sort_index()

print(f"\nPivot tables created: {price_pivot.shape[0]} dates x {price_pivot.shape[1]} tenors")

## 1. Term Structure Analysis

In [ ]:
# Calculate term structure metrics
def calculate_term_structure_metrics(price_data):
    """Calculate various term structure metrics"""
    metrics = pd.DataFrame(index=price_data.index)
    
    if 1 in price_data.columns and 3 in price_data.columns:
        metrics['M1_M3_Spread'] = price_data[3] - price_data[1]
    
    if 1 in price_data.columns and 6 in price_data.columns:
        metrics['M1_M6_Spread'] = price_data[6] - price_data[1]
        
    if 1 in price_data.columns and 12 in price_data.columns:
        metrics['M1_M12_Spread'] = price_data[12] - price_data[1]
    
    return metrics.dropna(how='all')

# Calculate metrics
ts_metrics = calculate_term_structure_metrics(price_pivot)

# Create visualization
fig = plt.figure(figsize=(16, 10))
gs = GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.3)

# 1. Calendar spreads
ax1 = fig.add_subplot(gs[0, :])
if 'M1_M3_Spread' in ts_metrics.columns:
    ax1.plot(ts_metrics.index, ts_metrics['M1_M3_Spread'], 
            color=COLORS['primary'], linewidth=2.5, label='M1-M3')
if 'M1_M6_Spread' in ts_metrics.columns:
    ax1.plot(ts_metrics.index, ts_metrics['M1_M6_Spread'], 
            color=COLORS['secondary'], linewidth=2.5, label='M1-M6')
if 'M1_M12_Spread' in ts_metrics.columns:
    ax1.plot(ts_metrics.index, ts_metrics['M1_M12_Spread'], 
            color=COLORS['tertiary'], linewidth=2.5, label='M1-M12')

ax1.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax1.set_title('Calendar Spreads Evolution', fontsize=14, weight='bold')
ax1.set_ylabel('Spread (USD/tonne)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
format_date_axis(ax1, data_range_days)

# 2. Front month price
ax2 = fig.add_subplot(gs[1, 0])
if 1 in price_pivot.columns:
    front_price = price_pivot[1].dropna()
    ax2.plot(front_price.index, front_price, color=COLORS['primary'], linewidth=2)
    
    # Add moving average
    ma_20 = front_price.rolling(20).mean()
    ax2.plot(ma_20.index, ma_20, color=COLORS['secondary'], 
            linewidth=1.5, alpha=0.8, label='20-day MA')
    
    ax2.set_title('Front Month Price', fontsize=14, weight='bold')
    ax2.set_ylabel('Price (USD/tonne)', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    format_date_axis(ax2, data_range_days)

# 3. Volatility
ax3 = fig.add_subplot(gs[1, 1])
if 1 in price_pivot.columns:
    returns = price_pivot[1].pct_change().dropna() * 100
    vol_20d = returns.rolling(20).std() * np.sqrt(252)
    
    ax3.plot(vol_20d.index, vol_20d, color=COLORS['quaternary'], linewidth=2)
    
    vol_mean = vol_20d.mean()
    ax3.axhline(vol_mean, color='green', linestyle='--', alpha=0.7, 
               label=f'Average: {vol_mean:.1f}%')
    
    ax3.set_title('20-Day Volatility', fontsize=14, weight='bold')
    ax3.set_ylabel('Volatility (%)', fontsize=12)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    format_date_axis(ax3, data_range_days)

plt.suptitle('Copper Futures Term Structure Analysis', fontsize=16, weight='bold')
plt.tight_layout()
plt.show()

# Summary
if not ts_metrics.empty:
    latest = ts_metrics.dropna().iloc[-1]
    print(f"\nLatest M1-M3 spread: ${latest.get('M1_M3_Spread', 0):.2f}/tonne")
    structure = 'Contango' if latest.get('M1_M3_Spread', 0) > 0 else 'Backwardation'
    print(f"Current structure: {structure}")

## 2. Liquidity Analysis

In [ ]:
# Liquidity metrics
def calculate_liquidity_metrics(volume_data):
    """Calculate liquidity metrics"""
    metrics = pd.DataFrame(index=volume_data.index)
    metrics['Total_Volume'] = volume_data.sum(axis=1, skipna=True)
    
    # Volume concentration in top 3 contracts
    metrics['Volume_Concentration'] = volume_data.apply(
        lambda x: (x.nlargest(3).sum() / x.sum()) if x.sum() > 0 else np.nan, axis=1
    )
    
    return metrics.dropna(how='all')

# Calculate metrics
liquidity_metrics = calculate_liquidity_metrics(volume_pivot)

# Create visualization
fig = plt.figure(figsize=(16, 8))
gs = GridSpec(1, 2, figure=fig, hspace=0.3, wspace=0.3)

# 1. Total volume
ax1 = fig.add_subplot(gs[0, 0])
if 'Total_Volume' in liquidity_metrics.columns:
    total_vol = liquidity_metrics['Total_Volume'].dropna()
    
    ax1.fill_between(total_vol.index, 0, total_vol, 
                    alpha=0.3, color=COLORS['primary'])
    ax1.plot(total_vol.index, total_vol, color=COLORS['primary'], linewidth=2)
    
    # Moving average
    vol_ma = total_vol.rolling(20).mean()
    ax1.plot(vol_ma.index, vol_ma, color=COLORS['secondary'], 
            linewidth=2, linestyle='--', label='20-day MA')
    
    ax1.set_title('Total Daily Volume', fontsize=14, weight='bold')
    ax1.set_ylabel('Volume (contracts)', fontsize=12)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    format_date_axis(ax1, data_range_days)

# 2. Volume concentration
ax2 = fig.add_subplot(gs[0, 1])
if 'Volume_Concentration' in liquidity_metrics.columns:
    conc_data = liquidity_metrics['Volume_Concentration'].dropna() * 100
    
    ax2.plot(conc_data.index, conc_data, 
            color=COLORS['tertiary'], linewidth=2)
    
    conc_mean = conc_data.mean()
    ax2.axhline(conc_mean, color='red', linestyle='--', alpha=0.7, 
               label=f'Average: {conc_mean:.1f}%')
    
    ax2.set_title('Volume Concentration (Top 3)', fontsize=14, weight='bold')
    ax2.set_ylabel('Concentration (%)', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    format_date_axis(ax2, data_range_days)

plt.suptitle('Copper Futures Liquidity Analysis', fontsize=16, weight='bold')
plt.tight_layout()
plt.show()

# Summary
if not liquidity_metrics.empty:
    latest = liquidity_metrics.dropna().iloc[-1]
    print(f"\nCurrent total volume: {latest.get('Total_Volume', 0):,.0f} contracts")
    print(f"Volume concentration: {latest.get('Volume_Concentration', 0)*100:.1f}%")

## 3. Performance and Risk Analysis

In [ ]:
# Performance metrics calculation
def calculate_performance_metrics(price_data):
    """Calculate performance metrics"""
    metrics = {}
    
    for tenor in price_data.columns:
        price_series = price_data[tenor].dropna()
        if len(price_series) > 20:
            returns = price_series.pct_change().dropna()
            
            metrics[f'M{int(tenor)}'] = {
                'Total_Return': (price_series.iloc[-1] / price_series.iloc[0] - 1) * 100,
                'Volatility': returns.std() * np.sqrt(252) * 100,
                'Sharpe_Ratio': (returns.mean() / returns.std()) * np.sqrt(252) if returns.std() > 0 else 0,
                'Max_Drawdown': calculate_max_drawdown(price_series),
                'VaR_95': np.percentile(returns, 5) * 100
            }
    
    return pd.DataFrame(metrics).T

def calculate_max_drawdown(price_series):
    """Calculate maximum drawdown"""
    cumulative = price_series
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max * 100
    return drawdown.min()

# Calculate metrics
performance_metrics = calculate_performance_metrics(price_pivot)

# Visualization
fig = plt.figure(figsize=(16, 10))
gs = GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.3)

# 1. Cumulative returns
ax1 = fig.add_subplot(gs[0, :])
colors = [COLORS['primary'], COLORS['secondary'], COLORS['tertiary'], COLORS['quaternary']]
key_tenors = [1, 3, 6, 12]

for i, tenor in enumerate(key_tenors):
    if tenor in price_pivot.columns:
        price_series = price_pivot[tenor].dropna()
        if len(price_series) > 1:
            cum_returns = (price_series / price_series.iloc[0] - 1) * 100
            ax1.plot(cum_returns.index, cum_returns, 
                    color=colors[i], linewidth=2.5, label=f'M{tenor}')

ax1.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax1.set_title('Cumulative Returns by Contract', fontsize=14, weight='bold')
ax1.set_ylabel('Cumulative Return (%)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
format_date_axis(ax1, data_range_days)

# 2. Risk-return scatter
ax2 = fig.add_subplot(gs[1, 0])
if not performance_metrics.empty:
    scatter = ax2.scatter(performance_metrics['Volatility'], 
                         performance_metrics['Total_Return'],
                         c=performance_metrics['Sharpe_Ratio'], 
                         cmap='RdYlGn', s=100, alpha=0.8, edgecolors='black')
    
    for idx, row in performance_metrics.iterrows():
        ax2.annotate(idx, (row['Volatility'], row['Total_Return']),
                    xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    plt.colorbar(scatter, ax=ax2, label='Sharpe Ratio')
    ax2.set_xlabel('Volatility (%)')
    ax2.set_ylabel('Total Return (%)')
    ax2.set_title('Risk-Return Profile', fontsize=14, weight='bold')
    ax2.grid(True, alpha=0.3)

# 3. Performance table
ax3 = fig.add_subplot(gs[1, 1])
ax3.axis('off')

if not performance_metrics.empty:
    # Create simplified table
    table_data = []
    for idx, row in performance_metrics.iterrows():
        table_data.append([
            idx,
            f"{row['Total_Return']:.1f}%",
            f"{row['Volatility']:.1f}%",
            f"{row['Sharpe_Ratio']:.2f}"
        ])
    
    table = ax3.table(cellText=table_data,
                     colLabels=['Contract', 'Return', 'Vol', 'Sharpe'],
                     cellLoc='center',
                     loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    
    ax3.set_title('Performance Summary', fontsize=14, weight='bold', pad=20)

plt.suptitle('Copper Futures Performance Analysis', fontsize=16, weight='bold')
plt.tight_layout()
plt.show()

# Performance summary
if not performance_metrics.empty:
    best_return = performance_metrics['Total_Return'].idxmax()
    best_sharpe = performance_metrics['Sharpe_Ratio'].idxmax()
    print(f"\nBest return: {best_return} ({performance_metrics.loc[best_return, 'Total_Return']:.1f}%)")
    print(f"Best Sharpe: {best_sharpe} ({performance_metrics.loc[best_sharpe, 'Sharpe_Ratio']:.2f})")

# Close connection
conn.close()
print("\nAnalysis complete. Database connection closed.")